In [1]:
import numpy as np
import matplotlib

import matplotlib.pyplot as plt
import psoap
from psoap.data import lkca14
from psoap import matrix_functions
from psoap import covariance

from matplotlib.ticker import FormatStrFormatter as FSF
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import MultipleLocator

In [2]:
order = 22
wl0 = 5165
wl1 = 5170

In [3]:
a = 0.26
l = 6.0

In [4]:
lkca14.sort_by_SN()
n_epochs = 3
wl = lkca14.wl[0:n_epochs,order,:]
# Sort by ind
ind = (wl[0] > wl0) & (wl[0] < wl1)

wl = wl[:,ind]
fl = lkca14.fl[0:n_epochs,order,ind]
sigma = lkca14.sigma[0:n_epochs,order,ind]
date1D = lkca14.date1D[0:n_epochs]

In [5]:
N = len(wl.flatten())
V11 = np.empty((N,N), dtype=np.float64)
matrix_functions.fill_V11_one(V11, wl.flatten(), a, l)
V11[np.diag_indices_from(V11)] += sigma.flatten()**2

In [26]:
fig, ax = plt.subplots(nrows=1, figsize=(3.5,3.5))
# ext = (wl0, wl1, wl0, wl1)
im = ax.imshow(V11, interpolation="none", cmap="viridis", origin="upper") #, extent=ext)
ax.set_xlabel(r"$\lambda\;[\AA]$")
ax.set_ylabel(r"$\lambda\;[\AA]$")

ax.xaxis.set_ticklabels([])
ax.yaxis.set_ticklabels([])
# ax.xaxis.set_major_locator(MultipleLocator(2.0))
# ax.yaxis.set_major_locator(MultipleLocator(2.0))

de = N/n_epochs

x_offset = 0.75 * de
y_offset = 0.2 * de
for i in range(n_epochs):
    for j in range(n_epochs):
        ax.annotate("({},{})".format(j+1,i+1), (x_offset + i * de, y_offset + j * de), color="w", size="x-small", ha="center")

lw=0.3
ax.axhline(de, color="w", lw=lw)
ax.axhline(2 * de, color="w", lw=lw)
ax.axvline(de, color="w", lw=lw)
ax.axvline(2 * de, color="w", lw=lw)
        
# Create an axis that is the pixel index
# Then label (1,1) (2,3) etc. In the caption we can explain what these reference.


fig.colorbar(im)

fig.subplots_adjust(left=0.2, right=0.9)
fig.savefig("V11_multi_epoch.pdf")
fig.savefig("V11_multi_epoch.png")

/home/ian/.build/anaconda/lib/python3.5/site-packages/matplotlib/pyplot.py:516: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
